## code to create the person_id list

In [ ]:

person_id = {}
idx=0
for recipient_list in training_info['recipients'].values:
    for recipient in recipient_list.split():
        if '@' in recipient:
            if recipient in person_id.keys():
                pass
            else:
                person_id[recipient] = idx
                idx+=1

In [ ]:
for sender in training['sender']:
    if sender not in person_id.keys():
        print sender
        person_id[sender] = idx
        idx+=1

In [ ]:
with io.open('../data/person_id.txt', 'w', encoding='utf-8') as f:
    f.write(unicode(json.dumps(person_id, ensure_ascii=False)))

In [ ]:
def cosine_similarity(a, b):
    assert len(a) == len(b), 'vectors need to have the same size'
    cos_sim = a.dot(b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
def most_n_similar(idx, array_embedding, n):
    query_embed = array_embedding[idx]
    list_scores = {}
    for i in range(idx) + range(idx + 1, len(array_embedding)):
        list_scores[i] = cosine_similarity(query_embed, array_embedding[i])

    closest_n_idx = heapq.nlargest(n, list_scores, key=list_scores.get)
    
    return closest_n_idx, list_scores

## First version of the frequentist algorithm

In [ ]:
query_id = 5
all_mean_ap = []


for query_id in training_info.tail(1000).index.values:

    if query_id%100==0:
        print query_id
    # number of closest neighbors to collect recipients from:
    n = 30

    # for testing:
    mail = training_info['body'].values[query_id]
    mail_tfidf = tfidf.transform([mail])
    # ground truth for scoring
    ground_truth = training_info['recipients'][query_id].split()

    # All closest emails (we will select the ones from the sender later)
    closest_ids = most_similar_sklearn(array_embedding_sparse, mail_tfidf)

    query_mid = training_info['mid'][query_id]

    # find the sender from the query email
    sender = get_sender(query_mid, training)

    # find the closest emails to the query one, written by the sender
    closest_ids_per_sender = get_n_closest_emails(sender, n, closest_ids, training, training_info)

    # Create dictionnary of all recipient for the 30 most similar emails, and get frequency
    # For the moment it is only the brute frequency, maybe we could refine this by adding wheights according to the closseness of the email
    suggested_10_recipients = get_10_recipients(closest_ids_per_sender, training_info)

    # print
    # print suggested_10_recipients
    all_mean_ap.append(mean_ap(suggested_10_recipients, ground_truth))

In [ ]:
X_train_info.index = range(X_train_info.shape[0])
duplicate_test_set_ids = []
count=0
for mail in X_test_info['body'].values:
    count+=1
    if count%100==0:
        print count
    mail_tfidf = tfidf.transform([mail])
    similarities = cosine_similarity(array_embedding_sparse, mail_tfidf)
    
    closest_id = similarities[:,0].argsort()[::-1][0]
    duplicate_test_set_ids.append(closest_id)